In [12]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import linear_kernel
from nltk.corpus import stopwords

import json

In [13]:
data_path = '/Users/PC-SANDER/Desktop/RS_TESIS/'
sitios = data_path + 'sitios-descripcion.csv'
df_sitios = pd.read_csv(sitios, encoding ='latin1', delimiter = ';')

In [14]:
df_sitios.head()

,id,sitio,descripcion
0,1,PARQUE LA ROTONDA,El Parque La Rotonda es un parque de la capita...
1,2,PARQUE LAS VEGAS,El Parque Las Vegas es un parque de la capital...
2,3,JARDIN BOTANICO,El Jardín Botánico de Portoviejo es una zona p...
3,4,PARQUE EL MAMEY,El Parque Ecológico Mamey es una alternativa p...
4,5,LA BOCA,"Mangle rojo, blanco, negro, iguanero y piñuelo..."


In [15]:
cachedStopWords = stopwords.words("spanish")
cachedStopWords[:10]


['de', 'la', 'que', 'el', 'en', 'y', 'a', 'los', 'del', 'se']

In [16]:
#Definir un objeto vectorizador TF-IDF. Elimine todas las palabras de detención en español
tfidf = TfidfVectorizer(stop_words=cachedStopWords)
tfidf.fit(df_sitios.descripcion)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True,
                stop_words=['de', 'la', 'que', 'el', 'en', 'y', 'a', 'los',
                            'del', 'se', 'las', 'por', 'un', 'para', 'con',
                            'no', 'una', 'su', 'al', 'lo', 'como', 'más',
                            'pero', 'sus', 'le', 'ya', 'o', 'este', 'sí',
                            'porque', ...],
                strip_accents=None, sublinear_tf=False,
                token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
                vocabulary=None)

In [17]:
# Reemplazar NaN con una cadena vacía
df_sitios['descripcion'] = df_sitios['descripcion'].fillna('')

In [18]:
#Construya la matriz TF-IDF requerida ajustando y transformando los datos
tfidf_matrix = tfidf.fit_transform(df_sitios['descripcion'])
tfidf_matrix.shape

(5, 108)

In [19]:
# Calcular la matriz de similitud de coseno
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [20]:
#Construya un mapa inverso de índices y nombre del sitio
indices = pd.Series(df_sitios.index, index=df_sitios['sitio']).drop_duplicates()


In [21]:
# Función que toma el nombre del sitio como entrada y emite la mayoría de los sitios similares
def get_recommendations(title, cosine_sim=cosine_sim):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:4]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    #return df_sitios['sitio'].iloc[movie_indices] 
        
    print(df_sitios['sitio'].iloc[movie_indices])   
            
    #para mandar los sitios como json
    x=[0]
    print ('--- DATOS JSON ---')
    for x in movie_indices:
        #c=movie_indices[x]
        c=df_sitios['sitio'].iloc[x]
        a=json.dumps(c)
        x=x+1
        print(a)
        
    return
      
    

In [22]:
get_recommendations('PARQUE LAS VEGAS')

0    PARQUE LA ROTONDA 
3       PARQUE EL MAMEY
2       JARDIN BOTANICO
Name: sitio, dtype: object
--- DATOS JSON ---
"PARQUE LA ROTONDA "
"PARQUE EL MAMEY"
"JARDIN BOTANICO"
